## Loading all packages

In [2]:
# Reload all modules
%load_ext autoreload
%autoreload 2

# Other package imports
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# User packages.
from features.FitnessAnalysis import MultipleFitnessAnalysis
from features.RandomWalkAnalysis import MultipleRandomWalkAnalysis
from features.LandscapeAnalysis import LandscapeAnalysis

from cases.MW_setup import MW3, MW7
import numpy as np
from optimisation.model.population import Population
from sampling.RandomWalk import RandomWalk
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Evaluating populations

In [3]:
problem = MW7(n_dim=5)  # use default dimensionality.
n_variables = problem.dim

# Experimental setup of Alsouly
neighbourhood_size = 2 * n_variables + 1
num_steps = int(n_variables / neighbourhood_size * 10**3)
step_size = 0.2  # 2% of the range of the instance domain

# Bounds of the decision variables.
x_lower = problem.lb
x_upper = problem.ub
bounds = np.vstack((x_lower, x_upper))

num_samples = 30

# Run feature eval multiple times.
pops = []
rw = RandomWalk(bounds, num_steps, step_size)
for ctr in range(num_samples):
    # Simulate RW
    walk = rw._do(seed=ctr)

    # Generate neighbours.
    new_walk = rw.generate_neighbours_for_walk(walk, neighbourhood_size)

    pop = Population(problem, n_individuals=num_steps)
    pop.evaluate(new_walk)
    print(
        "Evaluated rank and crowding for RW population {} of {}".format(
            ctr + 1, num_samples
        )
    )
    pops.append(pop)

# Saving results to pickle file.
with open("data/{}_d2_pop_data.pkl".format(problem.problem_name), "wb") as outp:
    pickle.dump(pops, outp, -1)

AttributeError: 'MW7' object has no attribute 'n_var'

## Evaluating features over all populations

In [ ]:
# Reload all modules
%load_ext autoreload
%autoreload 2

# Loading results from pickle file.
with open("data/MW7_d2_pop_data.pkl", "rb") as inp:
    pops = pickle.load(inp)


# Global features.
global_features = MultipleFitnessAnalysis(pops)
global_features.eval_features_for_all_populations()

# Random walk features.
rw_features = MultipleRandomWalkAnalysis(pops)
rw_features.eval_features_for_all_populations()

## Plotting decision variables, objectives

In [ ]:
# # Plot objectives
# # Create a 5x6 grid of subplots
# fig, ax = plt.subplots(5, 6, figsize=(10, 10))
# for i in range(5):
#     for j in range(6):
#         pop = pops[i + j]

#         obj = pop.extract_obj()

#         # Plot the objectives on the current subplot.
#         ax[i, j].scatter(obj[:, 0], obj[:, 1], s=2, color="red")

# # Adjust subplot spacing
# plt.tight_layout()

# # Show the plot
# plt.show()

# # Plot decisions
# # Create a 5x6 grid of subplots
# fig, ax = plt.subplots(5, 6, figsize=(10, 10))
# for i in range(5):
#     for j in range(6):
#         pop = pops[i + j]

#         var = pop.extract_var()

#         # Plot the varision vars on the current subplot.
#         ax[i, j].scatter(var[:, 0], var[:, 1], s=2, color="green")

# # Adjust subplot spacing
# plt.tight_layout()

# # Show the plot
# plt.show()

# Plot the pairwise combinations of decision variables.
pops[0].var_scatterplot_matrix()

## Aggregating features

In [ ]:
# Combine all features.
landscape = LandscapeAnalysis(global_features, rw_features)
landscape.extract_feature_arrays()
landscape.aggregate_features(YJ_transform=False)
landscape.extract_features_vector()
landscape.map_features_to_instance_space()

## Feature distributions

In [ ]:
%autoreload 2
landscape.plot_feature_histograms()
landscape.plot_feature_violin_box_plots()

## Comparison to Alsouly's results

In [ ]:
# Saving results to pickle file.
with open("data/MW3_landscape_data.pkl", "wb") as outp:
    pickle.dump(landscape, outp, -1)

# Apply YJ transformation
# landscape.aggregate_features(True)

aggregated_table = landscape.make_aggregated_feature_table()
unaggregated_global_table = landscape.make_unaggregated_global_feature_table()
unaggregated_rw_table = landscape.make_unaggregated_rw_feature_table()

alsouly_table = landscape.extract_experimental_results()

landscape.aggregate_features(True)
aggregated_YJ_table = landscape.make_aggregated_feature_table()


comp_table = pd.concat(
    [
        alsouly_table.loc[:, alsouly_table.columns != "feature_D"],
        aggregated_table,
        aggregated_YJ_table,
    ]
)

# Reset the index if needed
comp_table.reset_index(drop=True, inplace=True)
comp_table = comp_table.transpose()
print(comp_table)